<a href="https://colab.research.google.com/github/alecseiterr/safe_city/blob/main/Anton_Shalin/Cut_frames_from_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 Подготовка датасета. Выдергивание нужных кадров из видео.

 * Просматриваем видео, фиксируем начало и конец фрагмента в секундах, где присутствуют интересующие нас кадры в csv файл с таким же именем как видео
 * Созданный csv файл положить туда же, где лежит видео.
 * Код вырезает кадры с падениями и сохраняет в jpg. Скачивает готовый архив картинок.

In [ ]:
!rm -r sample_data

In [ ]:
import cv2
import csv
import os
import numpy as np
from google.colab import files
import shutil

In [ ]:
# Подключаем Google-диск
from google.colab import drive
drive.mount('/content/drive')
my_drv_path = '/content/drive/MyDrive/'

Mounted at /content/drive


In [ ]:
# Открываем видеофайл
video_path = '/content/drive/MyDrive/UII/VID_20240121_162756.mp4'
#video_path = '/content/drive/MyDrive/UII/VID_20240121_162437.mp4'
#video_path = '/content/drive/MyDrive/UII/VID_20240121_163004.mp4'
cap = cv2.VideoCapture(video_path)
csv_path = os.path.splitext(video_path)[0] + '.csv'

# Чтение временных промежутков из CSV-файла
time_intervals = []
with open(csv_path, 'r') as file:
    reader = csv.reader(file, delimiter=';')
    for row in reader:
        start_time, end_time = map(int, row)
        time_intervals.append((start_time, end_time))

# Создаем директорию для сохранения кадров
output_directory = 'IMG_data'
os.makedirs(output_directory, exist_ok=True)

# Создаем список для хранения уникальных кадров
unique_frames = []

# Функция для вычисления хэша изображения
def dhash(image, hash_size=8):
    resized = cv2.resize(image, (hash_size + 1, hash_size))
    diff = resized[:, 1:] > resized[:, :-1]
    return sum([2 ** i for (i, v) in enumerate(diff.flatten()) if v])

# Функция для проверки схожести двух изображений
def is_similar(image1, image2, threshold=80):
    hash1 = dhash(image1)
    hash2 = dhash(image2)
    difference = bin(hash1 ^ hash2).count('1')
    similarity = (1 - difference / (8 * 8)) * 100
    return similarity >= threshold

frame_number = 0

# Обрабатываем видеофайл
fps = int(cap.get(cv2.CAP_PROP_FPS))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_number += 1

    # Проверяем, находится ли кадр в любом из временных промежутков
    if any(start_time * fps <= frame_number <= end_time * fps for start_time, end_time in time_intervals):
        # Берем 3 кадра в секунду (каждый 10-й кадр)
        if frame_number % (fps // 3) == 0:
            # Проверка на дубликаты
            is_duplicate = any(is_similar(frame, unique_frame) for unique_frame in unique_frames)
            if not is_duplicate:
                unique_frames.append(frame)
                output_filename = os.path.join(output_directory, f"frame_{frame_number}.jpg")
                if cv2.imwrite(output_filename, frame, [int(cv2.IMWRITE_JPEG_QUALITY), 90]):
                    print(f"Сохранен кадр: {output_filename}")
                else:
                    print(f"Ошибка при сохранении кадра: {output_filename}")

cap.release()

# Архивируем папку с изображениями
shutil.make_archive(output_directory, 'zip', output_directory)

# Скачиваем архив
files.download(output_directory + '.zip')

Сохранен кадр: IMG_data/frame_10.jpg
Сохранен кадр: IMG_data/frame_20.jpg
Сохранен кадр: IMG_data/frame_30.jpg
Сохранен кадр: IMG_data/frame_40.jpg
Сохранен кадр: IMG_data/frame_50.jpg
Сохранен кадр: IMG_data/frame_60.jpg
Сохранен кадр: IMG_data/frame_70.jpg
Сохранен кадр: IMG_data/frame_80.jpg
Сохранен кадр: IMG_data/frame_90.jpg
Сохранен кадр: IMG_data/frame_100.jpg
Сохранен кадр: IMG_data/frame_110.jpg
Сохранен кадр: IMG_data/frame_120.jpg
Сохранен кадр: IMG_data/frame_130.jpg
Сохранен кадр: IMG_data/frame_140.jpg
Сохранен кадр: IMG_data/frame_150.jpg
Сохранен кадр: IMG_data/frame_160.jpg
Сохранен кадр: IMG_data/frame_170.jpg
Сохранен кадр: IMG_data/frame_180.jpg
Сохранен кадр: IMG_data/frame_190.jpg
Сохранен кадр: IMG_data/frame_200.jpg
Сохранен кадр: IMG_data/frame_210.jpg
Сохранен кадр: IMG_data/frame_220.jpg
Сохранен кадр: IMG_data/frame_230.jpg
Сохранен кадр: IMG_data/frame_240.jpg
Сохранен кадр: IMG_data/frame_250.jpg
Сохранен кадр: IMG_data/frame_260.jpg
Сохранен кадр: IMG_da

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>